In [1]:
import os
import math
import numpy as np
from pyannote.audio import Pipeline, Inference, Model
from scipy.spatial.distance import cdist
from extract_audio import *
from sp_diarization import *
from rttm_to_csv import *
from split_audio import * 

/Users/dominicvalentino/Documents/Harvard/presidential_recognition/pr/lib/python3.9/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/Users/dominicvalentino/Documents/Harvard/presidential_recognition/pr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pl.read_csv("../data/ad_validation_initial.csv")

In [3]:
pres_embeddings_directory = "../data/speech_embedding_models"
split_audio_directory = "../data/ad_audio_split_testing/"
hf_auth = os.environ['HF_AUTH']

In [4]:
embeddings_dict = {}
for filename in os.listdir(pres_embeddings_directory):
    if filename.endswith(".npy"):
        file_path = os.path.join(pres_embeddings_directory, filename)
        try:
            # Read the numpy array from the file
            embeddings = np.load(file_path)
            # Extract the speaker ID from the filename (assuming the filename is in the format "speaker_id.npy")
            speaker_id = os.path.splitext(filename)[0]
            # Add the embeddings to the dictionary with speaker ID as the key
            embeddings_dict[speaker_id] = embeddings
        except Exception as e:
            print(f"Error reading embeddings from '{filename}': {str(e)}")

model = Model.from_pretrained("pyannote/embedding", use_auth_token=hf_auth)
inference = Inference(model, window="whole")

n = df.shape[0]
for pres in embeddings_dict:
    temp_column = []
    print(pres)
    for i in range(0, n):
        file_path = split_audio_directory + df[i, 'file_id'] + "_" + str(round(df[i, 'start'], 3)) + ".wav"
        temp_embedding = inference(file_path)
        distance = 1 - (np.matmul(temp_embedding, embeddings_dict[pres]) / (math.sqrt(np.matmul(temp_embedding, temp_embedding)) * math.sqrt(np.matmul(embeddings_dict[pres], embeddings_dict[pres])))) # cosine distance
        temp_column.append(distance)
    df = df.with_columns(pl.Series(name=pres, values=temp_column))

df.head()

Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/c6335d8f1cd77b30084387468a6cf26fea90009b/pytorch_model.bin`
Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/c6335d8f1cd77b30084387468a6cf26fea90009b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.0. Bad things might happen unless you revert torch to 1.x.
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.0. Bad things might happen unless you revert torch to 1.x.
obama
gore
other
biden
kerry
bush
trump
romney
multi
clinton
mccain


file_id,speaker_est,start,duration,end,speaker_val,notes,obama,gore,other,biden,kerry,bush,trump,romney,multi,clinton,mccain
str,str,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""pres_trimmed_i…","""SPEAKER_04""",3.879,5.195,9.074,"""other""",null,-0.002337,-0.12813,-0.074071,-0.096068,0.06008,0.018305,0.130929,0.011301,-0.015798,-0.004148,-0.049703
"""pres_trimmed_i…","""SPEAKER_02""",9.228,2.733,11.960999,"""other""",null,0.088856,0.118086,0.061464,0.258948,0.160521,0.173348,0.149768,0.021883,0.19822,0.057804,0.100658
"""pres_trimmed_i…","""SPEAKER_02""",13.166,3.311,16.477001,"""other""",null,0.016902,0.109829,0.040137,0.188172,0.05597,0.187984,0.174335,0.091358,0.169485,0.028696,0.194025
"""pres_trimmed_i…","""SPEAKER_04""",16.426,8.761,25.187,"""other""",null,0.193229,0.07819,0.202613,0.226703,0.301889,0.173276,0.231272,0.234215,0.34692,0.090822,0.247603
"""pres_trimmed_i…","""SPEAKER_01""",25.934,3.107,29.041,"""clinton""",null,0.017145,0.018516,0.274224,0.077648,0.049014,0.077589,0.152602,0.090006,0.190045,0.38378,0.107407


In [7]:
df.shape

(171, 18)

In [19]:
i = 1
file_path = split_audio_directory + df[i, 'file_id'] + "_" + str(round(df[i, 'start'], 3)) + ".wav"
print(file_path)
temp_embedding = inference(file_path)

../data/ad_audio_split_testing/pres_trimmed_incl_scene-P-1904-108497_9.228.wav
